In [1]:
import pandas as pd
import os
from pyDOE import *
import xarray as xr
import copy
import netCDF4
import json
import csv

## Download latest version of params file from google drive
* requires 'publishing' the google drive spreadsheet
* file > publish to web
* then it can be set up to continuously publish the spreadsheet to a stable url (with some latency, maybe 1-2 minutes)
* note that the first tab must be the sheet where the relevant information is located

In [2]:
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
cmd = 'curl -L '+data_url+' > params.csv' # need to add -L option to force redirects
os.system(cmd)

0

## Define a class for organizing parameter information

In [3]:
class ParamInfo(object):
    """
    Stores parameter information.
    """
    
    def __init__(self, name, loc, minval=None, maxval=None, defval=None, value=None):
        self._name = name # parameter name
        self._min = minval # minimum value
        self._max = maxval # maximum value
        self._default = defval # default value
        self._value = value # actual value to be used in a given ensemble member
        self._location = loc # location of parameter (params file or namelist)

    @property
    def name(self):
        return self._name

    @property
    def min(self):
        return self._min

    @property
    def max(self):
        return self._max
    
    @property
    def default(self):
        return self._default
    
    @property
    def value(self):
        return self._value
    
    @property
    def location(self):
        return self._location
    
    @name.setter
    def name(self, new_name):
        self._name = new_name
   
    @min.setter
    def min(self, new_min):
        self._min = new_min
        
    @max.setter
    def max(self, new_max):
        self._max = new_max
        
    @default.setter
    def default(self, new_def):
        self._default = new_def
        
    @value.setter
    def value(self, new_val):
        self._value = new_val
            
    def __repr__(self):
        return "%s:\n\tloc = %s\n\tdefault = %s\n\tmin = %s\n\tmax = %s\n\tvalue = %s" % (self.name, self.location, self.default, self.min, self.max, self.value)

In [ ]:
# TO DO: add these or other unit tests to ParamInfo class itself
# Check every time the code updates, or adding new functionality

# testing out the class/dictionary functionality
test_dict = {"P1": ParamInfo("P1", minval=0.0, maxval=1.0, defval=2.0, loc='N'),
             "P2": ParamInfo("P2", minval=[0,0,0,0,0], maxval=[100,100,100,100,100], defval=[0,1,2,3,4], loc='P'),
             "P3": ParamInfo("P3", minval="min", maxval="max", defval="value", loc='N'),
             "P4": ParamInfo("P4", loc='P')
            }

# example of adding a new parameter
test_dict["new_param"] = ParamInfo("new_param", 'N')

# example of setting the max value
test_dict["P4"].max = 200

# example of setting the value for a given ensemble member
test_dict["new_param"].value = 100

# look at the test dictionary
for key in test_dict:
    print(test_dict[key])

## Define a class for organizing ensemble members

In [4]:
class Member(object):
    """
    Stores and works with a bunch of ParamInfos.
    """

    def __init__(self, name, paraminfo):
        self._name = name
        self._paraminfo = paraminfo
        
    @property
    def name(self):
        return self._name
    
    @property
    def paraminfo(self):
        return self._paraminfo
    
    @name.setter
    def name(self, new_name):
        self._name = new_name
    
    def get_names(self):
        """
        Returns a list of parameter names.
        """
        
        names = []
        for param in self._paraminfo:
            names.append(self._paraminfo[param].name)
        return names
                 
    def write(self, sampling_protocol):
        """
        Writes files to disk for each member: param netcdf, namelist mods txt.
        """
        
        # TO DO: remove existing files from these folders before writing output?
        
        # generate file names based on member name
        i = int(self._name)
        self._paramkey = sampling_protocol+str(i+1).zfill(4)
        #self._pftfile = "../paramfiles/"+self._paramkey+".nc"
        #self._nlfile = "../namelist_mods/"+self._paramkey+".txt" 
        self._pftfile = "/glade/scratch/kdagon/CLM5PPE/paramfiles/"+self._paramkey+".nc"
        self._nlfile = "/glade/scratch/kdagon/CLM5PPE/namelist_mods/"+self._paramkey+".txt"
        
        # assign the basepftfile (this also happens in Ensemble class)
        #self._basepftfile = "../basecase/clm5_params.c200717.nc"
        self._basepftfile= '/glade/p/cgd/tss/people/oleson/modify_param/ctsm51_params.c210217_kwo.c210222.nc'
        
        # create the pftfile as a copy of the basepftfile
        # NOTE: this will create a file for each ensemble member, regardless of if param mods are needed
        cmd = 'cp '+self._basepftfile+' '+self._pftfile
        os.system(cmd)
        
        # create the nlfile
        # NOTE: this will create a file for each ensemble member, regardless of if nl mods are needed
        with open(self._nlfile,"w") as file:
            output = "! user_nl_clm namelist options written by generate_params:\n"
            file.write(output)
            
        # read in the pftfile using netCDF4 package
        dset = netCDF4.Dataset(self._pftfile,'r+')
        
        # modify the param/nl files
        if sampling_protocol == "OAAT":
            for paramname in self.get_names():
                
                # TO DO: code these flags more universally?
                # NOTE: this assumes all flagged parameters are loc=P (true for now)
                
                # CWD flag 
                if paramname == ['cwd_fcel', 'cwd_flig']: 
                    # only modify if list is not empty
                    if self._paraminfo['CWD'].value:
                        # loop over parameters
                        for i in np.arange(len(paramname)):
                            print(paramname[i]+' modified in '+self._pftfile)
                            dset[paramname[i]][:] = self._paraminfo['CWD'].value[i]
                                
                # Q10 flag
                elif paramname == ['q10_hr', 'froz_q10']: 
                    # only modify if list is not empty
                    if self._paraminfo['Q10'].value:
                        # loop over parameters
                        for i in np.arange(len(paramname)):
                            print(paramname[i]+' modified in '+self._pftfile)
                            dset[paramname[i]][:] = self._paraminfo['Q10'].value[i]
                
                # LF flag
                elif paramname == ['lf_fcel', 'lf_flab', 'lf_flig']:
                    # only modify if list is not empty
                    if self._paraminfo['LF'].value:
                        # loop over parameters
                        for i in np.arange(len(paramname)):
                            print(paramname[i]+' modified in '+self._pftfile)
                            dset[paramname[i]][:] = self._paraminfo['LF'].value[i]
                
                # FR flag
                elif paramname == ['fr_fcel', 'fr_flab', 'fr_flig']:
                    # only modify if list is not empty
                    if self._paraminfo['FR'].value:
                        # loop over parameters
                        for i in np.arange(len(paramname)):
                            print(paramname[i]+' modified in '+self._pftfile)
                            dset[paramname[i]][:] = self._paraminfo['FR'].value[i]
                            
                # KCN flag
                elif paramname == ['kc_nonmyc', 'kn_nonmyc', 'akc_active', 'akn_active', 'ekc_active', 'ekn_active']:
                    # only modify if list is not empty
                    if self._paraminfo['KCN'].value:
                        # loop over parameters
                        for i in np.arange(len(paramname)):
                            print(paramname[i]+' modified in '+self._pftfile)
                            dset[paramname[i]][:] = self._paraminfo['KCN'].value[i]
                
                # for OAAT, only modify if value is not 'None'
                elif self._paraminfo[paramname].value is not None:
                    # params file
                    if self._paraminfo[paramname].location == "P":
                        print(paramname+' modified in '+self._pftfile)
                        dset[paramname][:] = self._paraminfo[paramname].value
                    # namelist
                    elif self._paraminfo[paramname].location == "N":
                        print(paramname+' modified in '+self._nlfile)
                        with open(self._nlfile,"a") as file: # using "a" for append option
                            output = "%s=%s\n" % (paramname, self._paraminfo[paramname].value) # TO DO: round values?
                            file.write(output)

        # TO DO: LHC code for writing files
        elif sampling_protocol == "LHC":
            pass
        
        # need to "close" netcdf file to write out
        dset.close() 
        
    def __repr__(self):
        i = int(self._name)
        return "Ensemble member %s" % (str(i+1), )

In [ ]:
# TO DO: add these or other unit tests to Member class itself
# Check every time the code updates, or adding new functionality

# testing out the class/dictionary functionality
member_test_dict = {"M1": Member("0", paraminfo=test_dict),
             "M2": Member("1", paraminfo=test_dict),
             "M3": Member("2", paraminfo=None),
             "M4": Member("3", paraminfo=None)
            }

# example of adding a new member
member_test_dict["new_member"] = Member("new_member", paraminfo=test_dict)

# example of setting the name
member_test_dict["new_member"].name = "4"

# look at the test dictionary
for key in member_test_dict:
    print(member_test_dict[key])
    
# look at a member's paraminfo in the test dictionary
member_test_dict['M1'].paraminfo

# look at a member's paraminfo for a specific parameter
member_test_dict['M1'].paraminfo['P1']

# get the list of parameter names
member_test_dict['M1'].get_names()

## Define a class for organizing the ensemble

In [36]:
class Ensemble(object):
    """
    Stores and works with a bunch of Members.
    """
    
    # assign the basepftfile (this also happens in Member class)
    #_basepftfile = "../basecase/clm5_params.c200717.nc"
    _basepftfile='/glade/p/cgd/tss/people/oleson/modify_param/ctsm51_params.c210217_kwo.c210222.nc'
    
    # using an example lnd_in file to pull in default namelist values
    # could also parse the namelist defaults file, see: https://github.com/ESCOMP/CTSM/blob/e2b9745d81ed5cb7cd7f5d6098edf506a4956335/bld/namelist_files/namelist_defaults_ctsm.xml
    #_thedir = '/glade/work/djk2120/ctsm_hardcode_co/cime/scripts/clm50c6_ctsmhardcodep_2deg_GSWP3V1_Sparse250_2000/CaseDocs/'
    _thedir = '/glade/work/oleson/lmbirch_wkattge.n01_ctsm5.1.dev006/cime/scripts/clm51_lmbirchwkattgen01ctsm51d006_2deg_GSWP3V1_PPE_1850pAD/CaseDocs/'
    _thefil = 'lnd_in'
    _lndin = _thedir+_thefil
    
    def __init__(self, csvfile, sampling_protocol):
        self._sampling_protocol = sampling_protocol
        
        print("Reading in data from spreadsheet...",end="")
        
        # read in csv file, filtering by the "include" column
        data = pd.read_csv(csvfile,header=0,skiprows=[1]) # modify read_csv to account for header spanning 2 rows
        #included = data['test'] == 1 # testing flag
        included = data['final'] == 1 # final flag
        params_full = data.loc[included,['name','location','min','max','flag','pft_mins','pft_maxs']]

        # reset indexing and get rid of excel row number
        params = params_full.reset_index(drop=True)
        
        print("Done")
        
        print("Creating dictionary of parameter information and reading default values...",end="")
        
        # declare a dictionary to store parameter information
        self._params_dict = {}
        
        # read in default pftfile
        #def_params = xr.open_dataset(self._basepftfile) 
        # replacing xarray with netcdf due to issues with reading ndays* params
        def_params = netCDF4.Dataset(self._basepftfile,'r')
        
        # assigning the default values
        # loop over parameters grabbing name, location, and flags
        for name,loc,flag in zip(params['name'],params['location'],params['flag']):      
            
            # check for flags
            # NOTE: this assumes all flagged parameters are loc=P (true for now)
            if pd.notnull(flag):

                # check to see if a params_dict entry has been created with flag label
                if flag in self._params_dict.keys():
                    # if it has already been created, append next parameter name and default value
                    self._params_dict[flag].name.append(name)
                    #x = def_params[name].values
                    x = def_params[name][:]
                    self._params_dict[flag].default.append(x) 
                else:
                    # if it hasn't been created, initialize the ParamInfo with placeholder lists
                    #x = def_params[name].values
                    x = def_params[name][:]
                    self._params_dict[flag] = ParamInfo(name=[name], loc=loc, defval=[x], minval=[], maxval=[], value=[])
                    
                # for KCN flag, parameters needs to be perturbed together and OAAT so also set up the OAAT ParamInfos
                if flag=='KCN':
                    #x = def_params[name].values
                    x = def_params[name][:]
                    self._params_dict[name] = ParamInfo(name, defval=x, loc=loc)    
            
            # select parameters located in the params file
            elif loc=='P':
                # getting parameter dims (i.e., checking for segment variation)
                #dims = len(def_params[name].values.shape)
                dims = len(def_params[name][:].shape)
                if dims<2:
                    # no segment variation
                    #x = def_params[name].values
                    x = def_params[name][:]
                else:
                    # segment variation: ck,kmax,psi50,rootprof_beta
                    # assumes the same values are applied across segments
                    # NOTE: this assumption is not strictly true for rootprof_beta
                    #x = def_params[name][0,:].values
                    x = def_params[name][0,:]
                    
                self._params_dict[name] = ParamInfo(name, defval=x, loc='P')
            
            # select namelist parameters
            elif loc=='N':
                # build a command to search lnd_in file for the parameter by name and put output in a tmp file
                cmd = 'grep '+name+' '+self._lndin+' > tmp.txt'
                ret = os.system(cmd)
                # checking for nonzero return code, meaning parameter is not found
                if ret != 0:
                    # NOTE: will need to address these special cases if they exist
                    print(name+' not found')
                else:
                    f = open('tmp.txt', 'r')
                    # parse the value from the parameter name
                    tmp = f.read().split()[2]
                    f.close()
                    # cases where scientific notation is specified by a "d"
                    if 'd' in tmp:
                        tmp = tmp.split('d')
                        x = float(tmp[0])*10**float(tmp[1])
                    else:
                        x = float(tmp)
                    self._params_dict[name] = ParamInfo(name, defval=x, loc='N')
                    # TO DO: remove tmp.txt file when finished?
        
        # count entries in params_dict as number of parameters (accounts for parameter dependencies)
        self._nparam = len(self._params_dict)
        
        print("Done")
        
        print("Assigning min and max values...",end="")
        
        # assigning min and max values
        if sampling_protocol == 'OAAT':
            for name,minv,maxv,pftmin,pftmax,flag in zip(params['name'],params['min'],params['max'],params['pft_mins'],params['pft_maxs'],params['flag']): 
                
                # first check for KCN flag, needs to be perturbed together and OAAT
                if flag=='KCN':
                    # check for "XXpercent" perturb from default
                    if "percent" in minv and "percent" in maxv: # NOTE: assumes "percent" is written in both min and max cells
                        percent_perturb_min = float(minv.split("percent")[0])
                        percent_perturb_max = float(maxv.split("percent")[0])
                        percent_min_values = self._params_dict[name].default*(1 - percent_perturb_min/100)
                        percent_max_values = self._params_dict[name].default*(1 + percent_perturb_max/100)
                        self._params_dict[name].min = percent_min_values
                        self._params_dict[name].max = percent_max_values
                    else:
                        self._params_dict[name].min = minv
                        self._params_dict[name].max = maxv
                
                # check for flags
                if pd.notnull(flag):
                    # check for "XXpercent" perturb from default
                    if "percent" in minv and "percent" in maxv: # NOTE: assumes "percent" is written in both min and max cells
                        # need to keep track parameter order for the default indexing
                        # only want to do this for loop ONCE so only execute if min/max lists are currently empty
                        if self._params_dict[flag].min == [] and self._params_dict[flag].max == []:
                            for i,p in enumerate(self._params_dict[flag].name):
                                percent_perturb_min = float(minv.split("percent")[0])
                                percent_perturb_max = float(maxv.split("percent")[0])
                                percent_min_values = self._params_dict[flag].default[i]*(1 - percent_perturb_min/100)
                                percent_max_values = self._params_dict[flag].default[i]*(1 + percent_perturb_max/100)
                                self._params_dict[flag].min.append(percent_min_values)
                                self._params_dict[flag].max.append(percent_max_values)
                    else:
                        # append min and max values for each parameter in a given flag
                        self._params_dict[flag].min.append(minv)
                        self._params_dict[flag].max.append(maxv)
                
                # check for pft variation
                elif minv=='pft' and maxv=='pft': # NOTE: assumes "pft" is written in both min and max cells
                    self._params_dict[name].min = np.fromstring(pftmin, dtype='float', sep=',')
                    self._params_dict[name].max = np.fromstring(pftmax, dtype='float', sep=',')
                
                # check for "XXpercent" perturb from default
                elif "percent" in minv and "percent" in maxv: # NOTE: assumes "percent" is written in both min and max cells
                    percent_perturb_min = float(minv.split("percent")[0])
                    percent_perturb_max = float(maxv.split("percent")[0])
                    percent_min_values = self._params_dict[name].default*(1 - percent_perturb_min/100)
                    percent_max_values = self._params_dict[name].default*(1 + percent_perturb_max/100)
                    self._params_dict[name].min = percent_min_values
                    self._params_dict[name].max = percent_max_values
                
                else:
                    # assign min/max values directly
                    self._params_dict[name].min = minv
                    self._params_dict[name].max = maxv
        
        elif sampling_protocol == 'LHC':
            # TO DO: assign LHC min/maxes
            pass
        
        print("Done")
        
        # declare a dictionary to store member information
        self._members = {}
        
        # set the number of ensemble members
        if sampling_protocol == 'OAAT':
            # number of samples is twice the number of parameter entries in param_dict (min and max perturbations)
            nsamp = 2*self._nparam
        elif sampling_protocol == 'LHC':
            # define sample size for LHC (user-specified)
            nsamp = 10
        
        print("Creating ensemble with "+str(nsamp)+" members...",end="")
        
        # create the ensemble members
        # need to "deepcopy" the dictionary for each member so they can be modified independently
        for i in range(nsamp):
            self._members[i] = Member(str(i), copy.deepcopy(self._params_dict))
        
        # loop over members and calculate parameter values for each member                
        # TO DO: OAAT ONLY - NEED TO ADD LHC CODE
        doneparams = []
        for member in self._members:
            for paramname in self._params_dict.keys():
                # check if this parameter has already been assigned
                if paramname in doneparams:
                    continue
                # check if this is a min or max perturbation
                if int(self._members[member].name)%2 == 0:
                    # min values
                    self._members[member].paraminfo[paramname].value = self._members[member].paraminfo[paramname].min
                    break
                else:
                    # max values
                    self._members[member].paraminfo[paramname].value = self._members[member].paraminfo[paramname].max
                    
                    # parameter is "done" after creating min/max ensemble members in sequence
                    doneparams.append(paramname)
                    break
        
        print("Done")
        
    @property
    def params(self):
        return self._params_dict
    
    @property
    def members(self):
        return self._members
    
    def output_files(self):
        """
        Loop over members in the ensemble and call the write function.
        """
        
        print("Writing files to disk for each member...")
        
        for member in self._members:
            self._members[member].write(self._sampling_protocol)
    
        print("Done")
    
    def save_psets(self, ensemble_name):
        """
        Save the parameter values for the ensemble.
        """

        print("Writing out parameter info for the ensemble...",end="")
        
        # build the file name with the prefix (ensemble type)
        #psetsfile = "../parameter_sets/"+self._sampling_protocol+"_"+ensemble_name+".txt"
        #psetsfile = "../parameter_sets/"+self._sampling_protocol+"_"+ensemble_name+".json"
        psetsfile = "../parameter_sets/"+self._sampling_protocol+"_"+ensemble_name+".csv"
        #psetsfile = "/glade/scratch/kdagon/CLM5PPE/parameter_sets/"+self._sampling_protocol+"_"+ensemble_name+".csv"
        
        # TO DO: figure out how to organize pset info
        # Question is for reading this info back in for analysis, what is the best way to store the pset info?
        
        # This writes a very long text file, but all the information is there:
        #with open(psetsfile, 'w') as fp:
            #for member in self._members:
                #fp.write(str(self._members[member]) + '\n')
                ##for paramname in self._members[member].get_names():
                #for paramname in self._params_dict.keys():
                    #fp.write(str(self._members[member].paraminfo[paramname]) + '\n')
                    #fp.write('\n')
                #fp.write('\n')
        
        # This writes a csv file with basic information:
        with open(psetsfile, 'w') as fp:
            pset_writer = csv.writer(fp, delimiter=',')
            
            # first row points to default file
            pset_writer.writerow([self._sampling_protocol+'0000', 'default', 'none', 'none'])
            
            # each subsequent row is an ensemble member
            for member in self._members:
                i = int(self._members[member].name)
                paramkey = self._sampling_protocol+str(i+1).zfill(4)
                
                # check for min or max
                if i%2 == 0:
                    # min values
                    perturblev = "min"
                else:
                    # max values
                    perturblev = "max"
                
                # write the specific paramname and min/max for each member
                # TO DO: code these flags more universally? see also Member class 'write' function
                # TO DO: add perturbation value at least for single-values, TBD PFT-dependencies
                for paramname in self._members[member].get_names():
                    if paramname == ['cwd_fcel', 'cwd_flig']: 
                        if self._members[member].paraminfo['CWD'].value:
                            paramval = self._members[member].paraminfo['CWD'].value
                            pset_writer.writerow([paramkey, paramname, perturblev, paramval])
                    elif paramname == ['q10_hr', 'froz_q10']:
                        if self._members[member].paraminfo['Q10'].value:
                            paramval = self._members[member].paraminfo['Q10'].value
                            pset_writer.writerow([paramkey, paramname, perturblev, paramval])
                    elif paramname == ['lf_fcel', 'lf_flab', 'lf_flig']:
                        if self._members[member].paraminfo['LF'].value:
                            paramval = self._members[member].paraminfo['LF'].value
                            pset_writer.writerow([paramkey, paramname, perturblev, paramval])
                    elif paramname == ['fr_fcel', 'fr_flab', 'fr_flig']:
                        if self._members[member].paraminfo['FR'].value:
                            paramval = self._members[member].paraminfo['FR'].value
                            pset_writer.writerow([paramkey, paramname, perturblev, paramval])
                    elif paramname == ['kc_nonmyc', 'kn_nonmyc', 'akc_active', 'akn_active', 'ekc_active', 'ekn_active']:
                        if self._members[member].paraminfo['KCN'].value:
                            paramval = self._members[member].paraminfo['KCN'].value
                            pset_writer.writerow([paramkey, paramname, perturblev, paramval])
                    elif self._members[member].paraminfo[paramname].value is not None:
                        paramval = self._members[member].paraminfo[paramname].value
                        pset_writer.writerow([paramkey, paramname, perturblev, paramval])
        
        print("Done")

## Use the Ensemble class with a given input csv file

In [37]:
# Using this Ensemble class with the input csv file
newEns = Ensemble('params.csv', 'OAAT')

Reading in data from spreadsheet...Done
Creating dictionary of parameter information and reading default values...Done
Assigning min and max values...Done
Creating ensemble with 400 members...Done


### The following includes checks and information associated with newEns

In [ ]:
# get the params_dict - not associated with a specific member (none of the "value" entries are set)
#newEns.params

# get parameter names for a specific ensemble member
#newEns.members[0].get_names()

# print member/param info across all members
#for member in newEns.members:
#    print(newEns.members[member])
#    print(newEns.members[member].paraminfo)
    
# print paraminfo for a given member
#newEns.members[0].paraminfo

# print paraminfo of a specific parameter for a given member
#newEns.members[0].paraminfo['fff']

# print paraminfo of a specific parameter for all members
#for member in newEns.members:
#    print(newEns.members[member].paraminfo['fff'])

# print the "value" info of a specific parameter for all members
#for member in newEns.members:
#    print(newEns.members[member].paraminfo['fff'].value)

In [38]:
# print list of parameters for a given member
# dependencies will be grouped together as a list
newEns.members[0].get_names()
# get number of parameters
len(newEns.members[0].get_names())

200

In [ ]:
# alternative way to print list of parameters
# dependencies will be denoted by the flag
#for param in newEns.members[0].paraminfo:
#    print(param)

In [ ]:
# sanity check the "values" across ensemble members
# NOTE: this may print a LOT of output if the parameter list is long
#for member in newEns.members:
#    print(newEns.members[member])
#    for param in newEns.members[member].paraminfo:
#        print(param)
#        print(newEns.members[member].paraminfo[param].value)

In [39]:
# sanity check length of pft-specific values
print(len(newEns.members[0].paraminfo['taulnir'].value))

79


## Write out paramfiles and nl_mod files

In [106]:
newEns.output_files()

Writing files to disk for each member...
taulnir modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0001.nc
taulnir modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0002.nc
taulvis modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0003.nc
taulvis modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0004.nc
tausnir modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0005.nc
tausnir modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0006.nc
tausvis modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0007.nc
tausvis modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0008.nc
rholnir modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0009.nc
rholnir modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0010.nc
rholvis modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0011.nc
rholvis modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0012.nc
rhosnir modified in /glade/scratch/kdagon/CLM5PPE/paramfiles/OAAT0013.nc
rhosnir mo

## Write out ensemble info

In [108]:
# Testing writing out ensemble info, need to provide a name for the Ensemble
#newEns.save_psets('test0001') # 148 parameters on 8/25/20
#newEns.save_psets('test0002') # 178 parameters on 10/9/20
#newEns.save_psets('test0003') # 183 parameters on 10/9/20
#newEns.save_psets('test0004') # 192 parameters on 10/14/20
#newEns.save_psets('test0005') # testing CWD dependency on 10/21/20
#newEns.save_psets('test0006') # testing all parameter dependencies on 10/21/20
#newEns.save_psets('test0007') # 192 parameters including dependencies on 10/22/20
#newEns.save_psets('test_miniens_20201102') # testing mini-ensemble (medlynslope and slatop) on 11/2/20
#newEns.save_psets('test_miniens_20201211') # testing mini-ensemble (sand_pf and clay_pf) on 12/11/20
#newEns.save_psets('test_bhs') # testing biomass heat storage params on 2/4/21
#newEns.save_psets('test_miniens_20210222') # testing mini-ensemble (17 params) on 2/22/21
#newEns.save_psets('20210408') # 400-member OAAT ensemble on 4/8/21
newEns.save_psets('csv_testing')

Writing out parameter info for the ensemble...Done


### Some notes on OAAT parameter perturbations:
* don't currently have checks to see if min or max are equivalent to default value (in that case, don't change params files / don't make a namelist mod?) - precision/round-off errors could be important here
* `rootprof_beta` has slighty different default values across variants, code currently assumes the same values applied across these secondary dims
* for parameters that vary with PFT but only a single value is specified in the spreadsheet, that value will propogate across all PFTs (including index 0 = non-vegetated)
* all current parameter dependencies (e.g., "these three terms need to add up to 1") are working, but solution could be cleaner and more universal

## (work in progress) Define a class for converting numpy arrays to be JSON serializable

In [ ]:
# Another option: Create a new dictionary converting numpy arrays to lists so they are serializable with JSON
# But then they are not readable as numpy arrays after saving out the JSON
        
# This JSON hack method doesn't work because self._members is a dictionary of dictionaries
#dumped = json.dumps(self._members, cls=NumpyEncoder)
#with open(psetsfile, 'w') as fp:
    #json.dump(dumped, fp)

class NumpyEncoder(json.JSONEncoder):
    """ 
    Special json encoder for numpy types
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

## TO DO: integrate this code above for LHC option
 * careful, each time you run LHC you get a new random draw

In [ ]:
if sampling_protocol == 'LHC':
    # define sample size (number of ensemble members)
    nsamp = 10

    # Generate the latin hypercube sample
    lhd = lhs(nparam, samples=int(nsamp))
    # lhd is a 2D array indexed by ensemble member x parameter
    
    # figure out how many pft-dependent params there are in this sample
    npftparam = sum(params['min']=='pft')
    
    if npftparam>0:
        # get dataframe index of first pft param
        pftfirstind = params.index[params['min']=='pft'][0]
        
        # get number of pfts
        npft = len(np.fromstring(params['pft_mins'][pftfirstind],dtype='float',sep=','))
        
        # set up numpy array to store pft-specific values
        pft_array = np.nan*np.ones([npftparam,npft,nsamp])
        
        for j in range(npftparam):
            # get the index for the current pft param
            pftind = params.index[params['min']=='pft'][j]
            
            # get min values
            min_pft_array = np.fromstring(params['pft_mins'][pftind],dtype='float',sep=',')
            # max values
            max_pft_array = np.fromstring(params['pft_maxs'][pftind],dtype='float',sep=',')
            
            # loop over samples and calculate parameter values for each pft
            for i in range(nsamp):
                pft_array[j,:,i] = (max_pft_array - min_pft_array)*lhd[i,pftind] + min_pft_array
                # can't store pft_array as a pandas dataframe because it's 3D
                # unless there is some alternate way to store this data?
    
    # initialize min/max arrays - for params without pft-variation
    min_array = np.nan*np.ones(nparam)
    max_array = np.nan*np.ones(nparam)
    
    # generate arrays with min and max values
    for i in range(nparam):
        if params['min'].values[i]=='pft':
            # TO DO: what's a good placeholder, to denote need to reference pft_array?
            # numpy doesn't like assigning a string to an existing array of floats
            # for now, just print a message
            print('skipping '+params['name'].values[i]+'...this parameter varies with PFT')
            
            # Numpy doesn't like assigning an array to a single index in an existing array
            # The problem is still that I'm declaring min_array before trying to assign values
            # If I could build it all at once, numpy would allow for nested arrays
            #min_array[i] = np.fromstring(params['pft_mins'].values[i],dtype='float',sep=',')
            #max_array[i] = np.fromstring(params['pft_maxs'].values[i],dtype='float',sep=',')
        else:
            # assign min/max values
            min_array[i] = float(params['min'].values[i])
            max_array[i] = float(params['max'].values[i])
            
    # calculate parameter values; skip pft params (NaNs in min/max arrays)
    param_array = (max_array - min_array)*lhd + min_array

# store psets in a pandas dataframe
#psets = pd.DataFrame(data=param_array, index=None, columns=params['name'])
#psets